In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, warnings, random, datetime

from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold, GroupKFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook
from tqdm import tqdm
import matplotlib.pylab as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.decomposition import PCA

import math
warnings.filterwarnings('ignore')

In [2]:
########################### Helpers
#################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
########################### Model
import lightgbm as lgb

def make_predictions(tr_df, tt_df, features_columns, target, lgb_params, NFOLDS=5):
    
    folds = GroupKFold(n_splits=NFOLDS)

    X,y = tr_df[features_columns], tr_df[target]    
    P,P_y = tt_df[features_columns], tt_df[target]  
    split_groups = tr_df['DT_M']

    tt_df = tt_df[['TransactionID',target]]    
    predictions = np.zeros(len(tt_df))
    oof = np.zeros(len(tr_df))
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y, groups=split_groups)):
        print('Fold:',fold_)
        tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
        vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]
            
        print(len(tr_x),len(vl_x))
        tr_data = lgb.Dataset(tr_x, label=tr_y)
        vl_data = lgb.Dataset(vl_x, label=vl_y)  

        estimator = lgb.train(
            lgb_params,
            tr_data,
            valid_sets = [tr_data, vl_data],
            verbose_eval = 200,
        )   
        
        pp_p = estimator.predict(P)
        predictions += pp_p/NFOLDS
        
        oof_preds = estimator.predict(vl_x)
        oof[val_idx] = (oof_preds - oof_preds.min())/(oof_preds.max() - oof_preds.min())

#         if LOCAL_TEST:
        feature_imp = pd.DataFrame(sorted(zip(estimator.feature_importance(),X.columns)), columns=['Value','Feature'])
        print(feature_imp)
        
        del tr_x, tr_y, vl_x, vl_y, tr_data, vl_data
        gc.collect()
        
    tt_df['prediction'] = predictions
    print('OOF AUC:', metrics.roc_auc_score(y, oof))
    if LOCAL_TEST:
        print('Holdout AUC:', metrics.roc_auc_score(tt_df[TARGET], tt_df['prediction']))
    
    return tt_df
## -------------------

In [4]:
########################### Vars
#################################################################################
SEED = 42
seed_everything(SEED)
LOCAL_TEST = False
TARGET = 'isFraud'
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

In [5]:
########################### Model params
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 2**8,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.5,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':800,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': SEED,
                    'early_stopping_rounds':100, 
                } 

In [6]:
########################### DATA LOAD
#################################################################################
print('Load Data')

if LOCAL_TEST:
    train_df = pd.read_pickle('../input/ieee-fe-for-local-test/train_df.pkl')
    test_df = pd.read_pickle('../input/ieee-fe-for-local-test/test_df.pkl') 
else:
    train_df = pd.read_pickle('../input/ieee-fe-with-some-eda/train_df.pkl')
    test_df = pd.read_pickle('../input/ieee-fe-with-some-eda/test_df.pkl')
    
remove_features = pd.read_pickle('../input/ieee-fe-with-some-eda/remove_features.pkl')
remove_features = list(remove_features['features_to_remove'].values)
print('Shape control:', train_df.shape, test_df.shape)

Load Data
Shape control: (590540, 791) (506691, 791)


In [7]:
# fe with amount (user id)
def fe_uid(train_df, test_df, rm_features):
    
    print('==> processing uid...')
    
    # user id, save for later aggregation
    for df in [train_df, test_df]:
        df['cents'] = df['TransactionAmt'].apply(lambda x: x - int(x))
        df['uid0'] = df['cents'].astype(str)+'_'+df['ProductCD'].astype(str)
        
        ### universal user
        # basic: uid2(card), uid3(addr), uid5(email)
        # aggr on uid4, uid5, uid7, uid8(*)
        # card
        df['uid1'] = df['card1'].astype(str)+'_'+df['card2'].astype(str)
        df['uid2'] = df['uid1'] + df['card3'].astype(str)+'_'+df['card5'].astype(str) # card holder
        
        # addr
        df['uid3'] =  df['addr1'].astype(str)+'_'+df['addr2'].astype(str) # family addr
        df['uid4'] =  df['uid2'].astype(str)+'_'+df['uid3'].astype(str)+'_'+df['ProductCD'].astype(str) # card holder with addr
                
        # email
        df['uid5'] = df['P_emaildomain'].astype(str)+'_'+df['R_emaildomain'].astype(str)
        df['uid6'] = df['uid2'].astype(str)+'_'+df['uid5'].astype(str)+'_'+df['ProductCD'].astype(str)
        df['uid7'] = df['uid3'].astype(str)+'_'+df['uid5'].astype(str)+'_'+df['ProductCD'].astype(str)
        
        # card + addr + email => specific user
        df['uid8'] = df['uid2'].astype(str)+'_'+df['uid3'].astype(str)+'_'+df['uid5'].astype(str)+'_'+df['ProductCD'].astype(str)

        ### uid4
        df['uid9'] = df['uid4'].astype(str)+'_'+df['M1'].astype(str)+'_'+df['M2'].astype(str)+'_'+df['M3'].astype(str)
        df['uid10'] = df['uid4'].astype(str)+'_'+df['M4'].astype(str)+'_'+df['M5'].astype(str)+'_'+df['M6'].astype(str)
        df['uid11'] = df['uid4'].astype(str)+'_'+df['M7'].astype(str)+'_'+df['M8'].astype(str)+'_'+df['M9'].astype(str)
        
        df['uid12'] = df['uid4'].astype(str)+'_'+df['C1'].astype(str)+'_'+df['C2'].astype(str)
        df['uid13'] = df['uid4'].astype(str)+'_'+df['C5'].astype(str)+'_'+df['C6'].astype(str)
        df['uid14'] = df['uid4'].astype(str)+'_'+df['C4'].astype(str)+'_'+df['C6'].astype(str)+'_'+df['C8'].astype(str)+'_'+df['C10'].astype(str)
        df['uid15'] = df['uid4'].astype(str)+'_'+df['C9'].astype(str)+'_'+df['C11'].astype(str)+'_'+df['C12'].astype(str)
        
        df['uid16'] = df['uid4'].astype(str)+'_'+df['D1'].astype(str)+'_'+df['D2'].astype(str)
        df['uid17'] = df['uid4'].astype(str)+'_'+df['D3'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D5'].astype(str)
        df['uid18'] = df['uid4'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D6'].astype(str)
        
        df['uid19'] = df['uid4'].astype(str)+'_'+df['D8'].astype(str)+'_'+df['D9'].astype(str)
        
        df['uid20'] = df['uid4'].astype(str)+'_'+df['D10'].astype(str)+'_'+df['D11'].astype(str)+'_'+df['D15'].astype(str)
        df['uid21'] = df['uid4'].astype(str)+'_'+df['D12'].astype(str)+'_'+df['D13'].astype(str)+'_'+df['D14'].astype(str)
        
        df['uid22'] = df['uid4'].astype(str)+'_'+df['dist1'].astype(str)+'_'+df['dist2'].astype(str)
        
        df['uid23'] = df['uid4'].astype(str)+'_'+df['id_01'].astype(str)+'_'+df['id_03'].astype(str)+'_'+df['id_04'].astype(str)
        df['uid24'] = df['uid4'].astype(str)+'_'+df['id_05'].astype(str)+'_'+df['id_06'].astype(str)+'_'+df['id_07'].astype(str)+'_'+df['id_08'].astype(str)
        df['uid25'] = df['uid4'].astype(str)+'_'+df['id_19'].astype(str)+'_'+df['id_20'].astype(str)
        df['uid26'] = df['uid4'].astype(str)+'_'+df['id_21'].astype(str)+'_'+df['id_22'].astype(str)+'_'+df['id_23'].astype(str)+'_'+df['id_24'].astype(str)+'_'+df['id_25'].astype(str)+'_'+df['id_26'].astype(str)+'_'+df['id_27'].astype(str) 
        df['uid27'] = df['uid4'].astype(str)+'_'+df['id_30'].astype(str)+'_'+df['id_31'].astype(str)+'_'+df['id_32'].astype(str)+'_'+df['id_33'].astype(str)+'_'+df['id_34'].astype(str)+'_'+df['id_35'].astype(str)+'_'+df['id_36'].astype(str) +df['id_37'].astype(str)+'_'+df['id_38'].astype(str)+'_'+df['DeviceInfo'].astype(str)+'_'+df['DeviceType'].astype(str)
        
        df['uid28'] = df['uid4'].astype(str)+'_'+df['TransactionAmt'].astype(str)
        
        df['uid29'] = df['uid4'].astype(str)+'_'+df['DT_W'].astype(str)
        df['uid30'] = df['uid4'].astype(str)+'_'+df['DT_D'].astype(str)
        df['uid31'] = df['uid4'].astype(str)+'_'+df['DT_day_week'].astype(str)
        df['uid32'] = df['uid4'].astype(str)+'_'+df['DT_day_month'].astype(str)
        df['uid33'] = df['uid4'].astype(str)+'_'+df['DT_hour'].astype(str)
        
        ### uid5
        df['uid34'] = df['uid5'].astype(str)+'_'+df['M1'].astype(str)+'_'+df['M2'].astype(str)+'_'+df['M3'].astype(str)
        df['uid35'] = df['uid5'].astype(str)+'_'+df['M4'].astype(str)+'_'+df['M5'].astype(str)+'_'+df['M6'].astype(str)
        df['uid36'] = df['uid5'].astype(str)+'_'+df['M7'].astype(str)+'_'+df['M8'].astype(str)+'_'+df['M9'].astype(str)
        
        df['uid37'] = df['uid5'].astype(str)+'_'+df['C1'].astype(str)+'_'+df['C2'].astype(str)
        df['uid38'] = df['uid5'].astype(str)+'_'+df['C5'].astype(str)+'_'+df['C6'].astype(str)
        df['uid39'] = df['uid5'].astype(str)+'_'+df['C4'].astype(str)+'_'+df['C6'].astype(str)+'_'+df['C8'].astype(str)+'_'+df['C10'].astype(str)
        df['uid40'] = df['uid5'].astype(str)+'_'+df['C9'].astype(str)+'_'+df['C11'].astype(str)+'_'+df['C12'].astype(str)
        
        df['uid41'] = df['uid5'].astype(str)+'_'+df['D1'].astype(str)+'_'+df['D2'].astype(str)
        df['uid42'] = df['uid5'].astype(str)+'_'+df['D3'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D5'].astype(str)
        df['uid43'] = df['uid5'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D6'].astype(str)
        
        df['uid44'] = df['uid5'].astype(str)+'_'+df['D8'].astype(str)+'_'+df['D9'].astype(str)
        
        df['uid45'] = df['uid5'].astype(str)+'_'+df['D10'].astype(str)+'_'+df['D11'].astype(str)+'_'+df['D15'].astype(str)
        df['uid46'] = df['uid5'].astype(str)+'_'+df['D12'].astype(str)+'_'+df['D13'].astype(str)+'_'+df['D14'].astype(str)
        
        df['uid47'] = df['uid5'].astype(str)+'_'+df['dist1'].astype(str)+'_'+df['dist2'].astype(str)
        
        df['uid48'] = df['uid5'].astype(str)+'_'+df['id_01'].astype(str)+'_'+df['id_03'].astype(str)+'_'+df['id_04'].astype(str)
        df['uid49'] = df['uid5'].astype(str)+'_'+df['id_05'].astype(str)+'_'+df['id_06'].astype(str)+'_'+df['id_07'].astype(str)+'_'+df['id_08'].astype(str)
        df['uid50'] = df['uid5'].astype(str)+'_'+df['id_19'].astype(str)+'_'+df['id_20'].astype(str)
        df['uid51'] = df['uid5'].astype(str)+'_'+df['id_21'].astype(str)+'_'+df['id_22'].astype(str)+'_'+df['id_23'].astype(str)+'_'+df['id_24'].astype(str)+'_'+df['id_25'].astype(str)+'_'+df['id_26'].astype(str)+'_'+df['id_27'].astype(str) 
        df['uid52'] = df['uid5'].astype(str)+'_'+df['id_30'].astype(str)+'_'+df['id_31'].astype(str)+'_'+df['id_32'].astype(str)+'_'+df['id_33'].astype(str)+'_'+df['id_34'].astype(str)+'_'+df['id_35'].astype(str)+'_'+df['id_36'].astype(str) +df['id_37'].astype(str)+'_'+df['id_38'].astype(str)+'_'+df['DeviceInfo'].astype(str)+'_'+df['DeviceType'].astype(str)
        
        df['uid53'] = df['uid5'].astype(str)+'_'+df['TransactionAmt'].astype(str)
        
        df['uid54'] = df['uid5'].astype(str)+'_'+df['DT_W'].astype(str)
        df['uid55'] = df['uid5'].astype(str)+'_'+df['DT_D'].astype(str)
        df['uid56'] = df['uid5'].astype(str)+'_'+df['DT_day_week'].astype(str)
        df['uid57'] = df['uid5'].astype(str)+'_'+df['DT_day_month'].astype(str)
        df['uid58'] = df['uid5'].astype(str)+'_'+df['DT_hour'].astype(str)
        
        ### uid7
        df['uid59'] = df['uid7'].astype(str)+'_'+df['M1'].astype(str)+'_'+df['M2'].astype(str)+'_'+df['M3'].astype(str)
        df['uid60'] = df['uid7'].astype(str)+'_'+df['M4'].astype(str)+'_'+df['M5'].astype(str)+'_'+df['M6'].astype(str)
        df['uid61'] = df['uid7'].astype(str)+'_'+df['M7'].astype(str)+'_'+df['M8'].astype(str)+'_'+df['M9'].astype(str)
        
        df['uid62'] = df['uid7'].astype(str)+'_'+df['C1'].astype(str)+'_'+df['C2'].astype(str)
        df['uid63'] = df['uid7'].astype(str)+'_'+df['C5'].astype(str)+'_'+df['C6'].astype(str)
        df['uid64'] = df['uid7'].astype(str)+'_'+df['C4'].astype(str)+'_'+df['C6'].astype(str)+'_'+df['C8'].astype(str)+'_'+df['C10'].astype(str)
        df['uid65'] = df['uid7'].astype(str)+'_'+df['C9'].astype(str)+'_'+df['C11'].astype(str)+'_'+df['C12'].astype(str)
        
        df['uid66'] = df['uid7'].astype(str)+'_'+df['D1'].astype(str)+'_'+df['D2'].astype(str)
        df['uid67'] = df['uid7'].astype(str)+'_'+df['D3'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D5'].astype(str)
        df['uid68'] = df['uid7'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D6'].astype(str)
        
        df['uid69'] = df['uid7'].astype(str)+'_'+df['D8'].astype(str)+'_'+df['D9'].astype(str)
        
        df['uid70'] = df['uid7'].astype(str)+'_'+df['D10'].astype(str)+'_'+df['D11'].astype(str)+'_'+df['D15'].astype(str)
        df['uid71'] = df['uid7'].astype(str)+'_'+df['D12'].astype(str)+'_'+df['D13'].astype(str)+'_'+df['D14'].astype(str)
        
        df['uid72'] = df['uid7'].astype(str)+'_'+df['dist1'].astype(str)+'_'+df['dist2'].astype(str)
        
        df['uid73'] = df['uid7'].astype(str)+'_'+df['id_01'].astype(str)+'_'+df['id_03'].astype(str)+'_'+df['id_04'].astype(str)
        df['uid74'] = df['uid7'].astype(str)+'_'+df['id_05'].astype(str)+'_'+df['id_06'].astype(str)+'_'+df['id_07'].astype(str)+'_'+df['id_08'].astype(str)
        df['uid75'] = df['uid7'].astype(str)+'_'+df['id_19'].astype(str)+'_'+df['id_20'].astype(str)
        df['uid76'] = df['uid7'].astype(str)+'_'+df['id_21'].astype(str)+'_'+df['id_22'].astype(str)+'_'+df['id_23'].astype(str)+'_'+df['id_24'].astype(str)+'_'+df['id_25'].astype(str)+'_'+df['id_26'].astype(str)+'_'+df['id_27'].astype(str) 
        df['uid77'] = df['uid7'].astype(str)+'_'+df['id_30'].astype(str)+'_'+df['id_31'].astype(str)+'_'+df['id_32'].astype(str)+'_'+df['id_33'].astype(str)+'_'+df['id_34'].astype(str)+'_'+df['id_35'].astype(str)+'_'+df['id_36'].astype(str) +df['id_37'].astype(str)+'_'+df['id_38'].astype(str)+'_'+df['DeviceInfo'].astype(str)+'_'+df['DeviceType'].astype(str)
        
        df['uid78'] = df['uid7'].astype(str)+'_'+df['TransactionAmt'].astype(str)
        
        df['uid79'] = df['uid7'].astype(str)+'_'+df['DT_W'].astype(str)
        df['uid80'] = df['uid7'].astype(str)+'_'+df['DT_D'].astype(str)
        df['uid81'] = df['uid7'].astype(str)+'_'+df['DT_day_week'].astype(str)
        df['uid82'] = df['uid7'].astype(str)+'_'+df['DT_day_month'].astype(str)
        df['uid83'] = df['uid7'].astype(str)+'_'+df['DT_hour'].astype(str)
        
        ### uid8
        df['uid84'] = df['uid8'].astype(str)+'_'+df['M1'].astype(str)+'_'+df['M2'].astype(str)+'_'+df['M3'].astype(str)
        df['uid85'] = df['uid8'].astype(str)+'_'+df['M4'].astype(str)+'_'+df['M5'].astype(str)+'_'+df['M6'].astype(str)
        df['uid86'] = df['uid8'].astype(str)+'_'+df['M7'].astype(str)+'_'+df['M8'].astype(str)+'_'+df['M9'].astype(str)
        
        df['uid87'] = df['uid8'].astype(str)+'_'+df['C1'].astype(str)+'_'+df['C2'].astype(str)
        df['uid88'] = df['uid8'].astype(str)+'_'+df['C5'].astype(str)+'_'+df['C6'].astype(str)
        df['uid89'] = df['uid8'].astype(str)+'_'+df['C4'].astype(str)+'_'+df['C6'].astype(str)+'_'+df['C8'].astype(str)+'_'+df['C10'].astype(str)
        df['uid90'] = df['uid8'].astype(str)+'_'+df['C9'].astype(str)+'_'+df['C11'].astype(str)+'_'+df['C12'].astype(str)
        
        df['uid91'] = df['uid8'].astype(str)+'_'+df['D1'].astype(str)+'_'+df['D2'].astype(str)
        df['uid92'] = df['uid8'].astype(str)+'_'+df['D3'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D5'].astype(str)
        df['uid93'] = df['uid8'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D6'].astype(str)
        
        df['uid94'] = df['uid8'].astype(str)+'_'+df['D8'].astype(str)+'_'+df['D9'].astype(str)
        
        df['uid95'] = df['uid8'].astype(str)+'_'+df['D10'].astype(str)+'_'+df['D11'].astype(str)+'_'+df['D15'].astype(str)
        df['uid96'] = df['uid8'].astype(str)+'_'+df['D12'].astype(str)+'_'+df['D13'].astype(str)+'_'+df['D14'].astype(str)
        
        df['uid97'] = df['uid8'].astype(str)+'_'+df['dist1'].astype(str)+'_'+df['dist2'].astype(str)
        
        df['uid98'] = df['uid8'].astype(str)+'_'+df['id_01'].astype(str)+'_'+df['id_03'].astype(str)+'_'+df['id_04'].astype(str)
        df['uid99'] = df['uid8'].astype(str)+'_'+df['id_05'].astype(str)+'_'+df['id_06'].astype(str)+'_'+df['id_07'].astype(str)+'_'+df['id_08'].astype(str)
        df['uid100'] = df['uid8'].astype(str)+'_'+df['id_19'].astype(str)+'_'+df['id_20'].astype(str)
        df['uid101'] = df['uid8'].astype(str)+'_'+df['id_21'].astype(str)+'_'+df['id_22'].astype(str)+'_'+df['id_23'].astype(str)+'_'+df['id_24'].astype(str)+'_'+df['id_25'].astype(str)+'_'+df['id_26'].astype(str)+'_'+df['id_27'].astype(str) 
        df['uid102'] = df['uid8'].astype(str)+'_'+df['id_30'].astype(str)+'_'+df['id_31'].astype(str)+'_'+df['id_32'].astype(str)+'_'+df['id_33'].astype(str)+'_'+df['id_34'].astype(str)+'_'+df['id_35'].astype(str)+'_'+df['id_36'].astype(str) +df['id_37'].astype(str)+'_'+df['id_38'].astype(str)+'_'+df['DeviceInfo'].astype(str)+'_'+df['DeviceType'].astype(str)
        
        df['uid103'] = df['uid8'].astype(str)+'_'+df['TransactionAmt'].astype(str)
        
        df['uid104'] = df['uid8'].astype(str)+'_'+df['DT_W'].astype(str)
        df['uid105'] = df['uid8'].astype(str)+'_'+df['DT_D'].astype(str)
        df['uid106'] = df['uid8'].astype(str)+'_'+df['DT_day_week'].astype(str)
        df['uid107'] = df['uid8'].astype(str)+'_'+df['DT_day_month'].astype(str)
        df['uid108'] = df['uid8'].astype(str)+'_'+df['DT_hour'].astype(str)
        
    uid_list = ['uid' + str(i) for i in range(109)]
    cid_list  =[]
            
    tmp_rm = [] + uid_list

    rm_features = rm_features + tmp_rm
    return train_df, test_df, rm_features, uid_list, cid_list
# train, test, rm_features = fe_uid(train, test, rm_features)

In [8]:
# group aggregation
def fe_agg(train_df, test_df, rm_features, uid_list, cid_list):

    print('==> processing aggregation...')

    uid_cols = uid_list
    
    for col in tqdm_notebook(uid_cols):
        # aggr: mean, std, min, max, sum
        for agg_type in ['mean', 'std']:
            for agg_col in ['TransactionAmt']:
                new_col_name = col + '_' + agg_col + '_' + agg_type + '_'
                temp_df = pd.concat(
                    [train_df[[col, agg_col]], test_df[[col, agg_col]]])
                temp_df = temp_df.groupby([col])[agg_col].agg([
                    agg_type
                ]).reset_index().rename(columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name] = test_df[col].map(temp_df)  
                
    for col in tqdm_notebook(uid_cols):
        # aggr: value - mean
        for agg_type in ['mean']:
            for agg_col in ['TransactionAmt']:
                new_col_name = col + '_' + agg_col + '_' + agg_type + '_diff'
                temp_df = pd.concat(
                    [train_df[[col, agg_col]], test_df[[col, agg_col]]])
                temp_df = temp_df.groupby([col])[agg_col].agg([
                    agg_type
                ]).reset_index().rename(columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name] = test_df[col].map(temp_df) 
                train_df[new_col_name] = train_df[agg_col] - train_df[new_col_name]
                test_df[new_col_name] = test_df[agg_col] - test_df[new_col_name]
                
    for col in tqdm_notebook(uid_cols + cid_list):
        # count
        for agg_type in ['count']:
            for agg_col in ['TransactionDT']:
                new_col_name = col + '_' + agg_type
                temp_df = pd.concat(
                    [train_df[[col, agg_col]], test_df[[col, agg_col]]])
                temp_df = temp_df.groupby([col])[agg_col].agg([
                    agg_type
                ]).reset_index().rename(columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name] = test_df[col].map(temp_df)
        
        
    train_df = train_df.replace(np.inf, 999)
    test_df = test_df.replace(np.inf, 999)
    
    tmp_rm = []
    rm_features = rm_features + tmp_rm
    return train_df, test_df, rm_features

# train, test, rm_features = fe_agg(train, test, rm_features)

In [9]:
def process_v(train_df, test_df, v_list):
    n_com = 5
    pca = PCA(n_components=n_com, random_state=42)
    fill_val = 0

    temp_df = pd.concat([train_df[v_list], test_df[v_list]])
    pca.fit_transform(temp_df[v_list].fillna(fill_val))
    train_compressed = pca.transform(train_df[v_list].fillna(fill_val))
    test_compressed = pca.transform(test_df[v_list].fillna(fill_val))

    for i in range(n_com):
        train_df[str(v_list[0]) + '_pca_' + str(i)] = train_compressed[:, i]
        test_df[str(v_list[0]) + '_pca_' + str(i)] = test_compressed[:, i]

    return train_df, test_df

In [10]:
# fe with V 1-339
def fe_v(train_df, test_df, rm_features):
    
    print('==> processing V...')
    
    v_cols = ['V' + str(i) for i in range(1,340)]
       
    # V 1-11
    v_1_11 = ['V' + str(i) for i in range(1,12)]
    train_df, test_df = process_v(train_df, test_df, v_1_11)
    
    # V 12-34
    v_12_34 = ['V' + str(i) for i in range(12,35)]
    train_df, test_df = process_v(train_df, test_df, v_12_34)
    
    # V 35-52
    v_35_52 = ['V' + str(i) for i in range(35,53)]
    train_df, test_df = process_v(train_df, test_df, v_35_52)
    
    # V 53-74
    v_53_74 = ['V' + str(i) for i in range(53,75)]
    train_df, test_df = process_v(train_df, test_df, v_53_74)
    
    # V 75-94
    v_75_94 = ['V' + str(i) for i in range(75,95)]
    train_df, test_df = process_v(train_df, test_df, v_75_94)
    
    # V 95-137
    v_95_137 = ['V' + str(i) for i in range(95,138)]
    train_df, test_df = process_v(train_df, test_df, v_95_137)
    
    # V 138-166
    v_138_166 = ['V' + str(i) for i in range(138,167)]
    train_df, test_df = process_v(train_df, test_df, v_138_166)
    
    # V 167-216
    v_167_216 = ['V' + str(i) for i in range(167,217)]
    train_df, test_df = process_v(train_df, test_df, v_167_216)
    
    # V 217-278
    v_217_278 = ['V' + str(i) for i in range(217,279)]
    train_df, test_df = process_v(train_df, test_df, v_217_278)
    
    # V 279-321
    v_279_321 = ['V' + str(i) for i in range(279,322)]
    train_df, test_df = process_v(train_df, test_df, v_279_321)
    
    # V 322-339
    v_322_339 = ['V' + str(i) for i in range(322,339)]
    train_df, test_df = process_v(train_df, test_df, v_322_339)
    
    tmp_rm = [] + v_cols
    rm_features = rm_features + tmp_rm
    return train_df, test_df, rm_features
# train, test, rm_features = fe_v(train, test, rm_features)

In [11]:
def fe(train_df, test_df, rm_features):
    
    #fe with amount (user id)
    train_df, test_df, rm_features, uid_list, cid_list = fe_uid(train_df, test_df, rm_features)

    # group aggregation
    train_df, test_df, rm_features = fe_agg(train_df, test_df, rm_features, uid_list, cid_list)
    
    # pca on v features
    train_df, test_df, rm_features = fe_v(train_df, test_df, rm_features)
    
    return train_df, test_df, rm_features

train_df, test_df, remove_features = fe(train_df, test_df, remove_features)

==> processing uid...
==> processing aggregation...



==> processing V...


In [12]:
print('Shape control:', train_df.shape, test_df.shape)
print(remove_features)

Shape control: (590540, 1384) (506691, 1384)
['TransactionID', 'TransactionDT', 'isFraud', 'DT', 'DT_M', 'DT_W', 'DT_D', 'DT_hour', 'DT_day_week', 'DT_day_month', 'DT_M_total', 'DT_W_total', 'DT_D_total', 'uid', 'uid2', 'uid3', 'uid4', 'uid5', 'bank_type', 'uid0', 'uid1', 'uid2', 'uid3', 'uid4', 'uid5', 'uid6', 'uid7', 'uid8', 'uid9', 'uid10', 'uid11', 'uid12', 'uid13', 'uid14', 'uid15', 'uid16', 'uid17', 'uid18', 'uid19', 'uid20', 'uid21', 'uid22', 'uid23', 'uid24', 'uid25', 'uid26', 'uid27', 'uid28', 'uid29', 'uid30', 'uid31', 'uid32', 'uid33', 'uid34', 'uid35', 'uid36', 'uid37', 'uid38', 'uid39', 'uid40', 'uid41', 'uid42', 'uid43', 'uid44', 'uid45', 'uid46', 'uid47', 'uid48', 'uid49', 'uid50', 'uid51', 'uid52', 'uid53', 'uid54', 'uid55', 'uid56', 'uid57', 'uid58', 'uid59', 'uid60', 'uid61', 'uid62', 'uid63', 'uid64', 'uid65', 'uid66', 'uid67', 'uid68', 'uid69', 'uid70', 'uid71', 'uid72', 'uid73', 'uid74', 'uid75', 'uid76', 'uid77', 'uid78', 'uid79', 'uid80', 'uid81', 'uid82', 'uid83

In [13]:
########################### Final features list
features_columns = [col for col in list(train_df) if col not in remove_features]

########################### Final Minification
## I don't like this part as it changes float numbers
## small change but change.
## To be able to train lgbm without 
## minification we need to do some changes on model
## we will do it later.
if not LOCAL_TEST:
    train_df = reduce_mem_usage(train_df)
    test_df  = reduce_mem_usage(test_df)

Mem. usage decreased to 2091.49 Mb (57.4% reduction)
Mem. usage decreased to 1807.63 Mb (57.2% reduction)


In [28]:
print('feature # used: ', len(features_columns))

feature # used:  921


In [15]:
train_df.head()

TransactionID  isFraud  TransactionDT  TransactionAmt  ProductCD    card1  \
0        2987000        0          86400        4.242188   0.020401  13928.0   
1        2987001        0          86401        3.400391   0.020401   2756.0   
2        2987002        0          86469        4.093750   0.020401   4664.0   
3        2987003        0          86499        3.931641   0.020401  18128.0   
4        2987004        0          86506        3.931641   0.047668   4496.0   

   card2  card3     card4  card5     card6  addr1  addr2  dist1  dist2  \
0    NaN  150.0    9524.0  142.0  267648.0  315.0   87.0   19.0    NaN   
1  404.0  150.0  347386.0  102.0  267648.0  325.0   87.0    NaN    NaN   
2  490.0  150.0  719649.0  166.0  824959.0  330.0   87.0  287.0    NaN   
3  567.0  150.0  347386.0  117.0  824959.0  476.0   87.0    NaN    NaN   
4  514.0  150.0  347386.0  102.0  267648.0  420.0   87.0    NaN    NaN   

   P_emaildomain  R_emaildomain   C1   C2   C3   C4   C5   C6   C7   C8   C9  \
0             49             49  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
1             16             49  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
2             35             49  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
3             55             49  2.0  5.0  0.0  0.0  0.0  4.0  0.0  0.0  1.0   
4             16             49  1.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0   

   C10  C11  C12   C13  C14      D1      D2      D3      D4      D5      D6  \
0  0.0  2.0  0.0   1.0  1.0    5548  515566   11525  245773  534216  899261   
1  0.0  1.0  0.0   1.0  1.0  507854  515566  466020  344864  534216  899261   
2  0.0  1.0  0.0   1.0  1.0  507854  515566  466020  344864  534216  899261   
3  0.0  1.0  0.0  25.0  1.0    1440    1374  146326     894  139264  899261   
4  1.0  1.0  0.0   1.0  1.0  507854  515566  466020  245773  534216  899261   

       D7      D8      D9     D10     D11     D12     D13     D14     D15  \
0  998181  947967  947967    3208    2304  963260  911895  919850  357981   
1  998181  947967  947967  449929  455805  963260  911895  919850  357981   
2  998181  947967  947967  449929     487  963260  911895  919850     683   
3  998181  947967  947967    1588  455805  963260  911895  919850     994   
4  998181  947967  947967   88567  455805  963260  911895  919850  101182   

    M1   M2   M3        M4   M5   M6   M7   M8   M9   V1   V2   V3   V4   V5  \
0  1.0  1.0  1.0  0.113770  0.0  1.0  NaN  NaN  NaN  1.0  1.0  1.0  1.0  1.0   
1  NaN  NaN  NaN  0.036652  1.0  1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2  1.0  1.0  1.0  0.036652  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0   
3  NaN  NaN  NaN  0.036652  1.0  0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4  NaN  NaN  NaN       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

    V6   V7   V8   V9  V10  V11  V12  V13  V14  V15  V16  V17  V18  V19  V20  \
0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0   
1  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0   
2  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0   
3  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0   
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   V21  V22  V23  V24  V25  V26  V27  V28  V29  V30  V31  V32  V33  V34  V35  \
0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  NaN   
1  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
3  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   V36  V37  V38  V39  V40  V41  V42  V43  V44  V45  V46  V47  V48  V49  V50  \
0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.

In [29]:
########################### Model Train
if LOCAL_TEST:
    lgb_params['learning_rate'] = 0.01
    lgb_params['n_estimators'] = 10000
    lgb_params['early_stopping_rounds'] = 100
    test_predictions = make_predictions(train_df, test_df, features_columns, TARGET, lgb_params, NFOLDS=4)
else:
    lgb_params['learning_rate'] = 0.01
    lgb_params['n_estimators'] = 2000
    lgb_params['early_stopping_rounds'] = 100    
    test_predictions = make_predictions(train_df, test_df, features_columns, TARGET, lgb_params, NFOLDS=6)

Fold: 0
453219 137321
Training until validation scores don't improve for 100 rounds.
[200]	training's auc: 0.981497	valid_1's auc: 0.895021
[400]	training's auc: 0.997149	valid_1's auc: 0.905652
[600]	training's auc: 0.999526	valid_1's auc: 0.9113
[800]	training's auc: 0.999911	valid_1's auc: 0.913967
[1000]	training's auc: 0.999988	valid_1's auc: 0.915645
[1200]	training's auc: 0.999999	valid_1's auc: 0.916665
[1400]	training's auc: 1	valid_1's auc: 0.917193
[1600]	training's auc: 1	valid_1's auc: 0.917587
Early stopping, best iteration is:
[1668]	training's auc: 1	valid_1's auc: 0.917583
     Value                             Feature
0        0                               id_22
1        0                               id_27
2        0                         is_december
3        0       uid0_TransactionAmt_mean_diff
4        0            uid0_TransactionAmt_std_
5        0     uid103_TransactionAmt_mean_diff
6        0      uid28_TransactionAmt_mean_diff
7        0      uid53_Trans

488908 101632
Training until validation scores don't improve for 100 rounds.
[200]	training's auc: 0.981472	valid_1's auc: 0.92787
[400]	training's auc: 0.997139	valid_1's auc: 0.939701
[600]	training's auc: 0.99956	valid_1's auc: 0.943406
[800]	training's auc: 0.99992	valid_1's auc: 0.944858
[1000]	training's auc: 0.999989	valid_1's auc: 0.945988
[1200]	training's auc: 0.999999	valid_1's auc: 0.946372
[1400]	training's auc: 1	valid_1's auc: 0.946526
[1600]	training's auc: 1	valid_1's auc: 0.946738
Early stopping, best iteration is:
[1613]	training's auc: 1	valid_1's auc: 0.946842
     Value                             Feature
0        0                               id_22
1        0                               id_27
2        0       uid0_TransactionAmt_mean_diff
3        0     uid103_TransactionAmt_mean_diff
4        0      uid28_TransactionAmt_mean_diff
5        0      uid53_TransactionAmt_mean_diff
6        0      uid78_TransactionAmt_mean_diff
7        1                          

497955 92585
Training until validation scores don't improve for 100 rounds.
[200]	training's auc: 0.979465	valid_1's auc: 0.925263
[400]	training's auc: 0.996922	valid_1's auc: 0.93722
[600]	training's auc: 0.99949	valid_1's auc: 0.94129
[800]	training's auc: 0.999899	valid_1's auc: 0.9431
[1000]	training's auc: 0.999985	valid_1's auc: 0.944179
[1200]	training's auc: 0.999999	valid_1's auc: 0.944576
Early stopping, best iteration is:
[1111]	training's auc: 0.999996	valid_1's auc: 0.944724
     Value                             Feature
0        0                           D9_not_na
1        0                               id_22
2        0                               id_27
3        0       uid0_TransactionAmt_mean_diff
4        0            uid0_TransactionAmt_std_
5        0     uid103_TransactionAmt_mean_diff
6        0      uid28_TransactionAmt_mean_diff
7        0      uid53_TransactionAmt_mean_diff
8        0      uid78_TransactionAmt_mean_diff
9        1                          

501214 89326
Training until validation scores don't improve for 100 rounds.
[200]	training's auc: 0.980337	valid_1's auc: 0.920522
[400]	training's auc: 0.996739	valid_1's auc: 0.933855
[600]	training's auc: 0.999464	valid_1's auc: 0.939283
[800]	training's auc: 0.999887	valid_1's auc: 0.94169


KeyboardInterrupt: 

In [ ]:
# ########################### Export
# if not LOCAL_TEST:
#     test_predictions['isFraud'] = test_predictions['prediction']
#     test_predictions[['TransactionID','isFraud']].to_csv('../submissions/submission_4.csv', index=False)